# Web Mining and Applied NLP (44-620)

## Web Scraping and NLP with Requests, BeautifulSoup, and spaCy

### Student Name: Joanna Farris

##### GitHub Repo: [https://github.com/Pojetta/module_06](https://github.com/Pojetta/module_06)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

In [1]:
import json  
import pathlib  
import pickle  
from collections import Counter  
  
import requests  
from bs4 import BeautifulSoup  
import matplotlib.pyplot as plt  
import spacy  
from spacy.tokens import Doc  
from spacytextblob.spacytextblob import SpacyTextBlob

In [2]:
nlp = spacy.load("en_core_web_sm")  
nlp.add_pipe("spacytextblob")

In [3]:
if not Doc.has_extension("polarity"):  
    Doc.set_extension("polarity", getter=lambda doc: doc._.blob.sentiment.polarity)  
  
if not Doc.has_extension("subjectivity"):  
    Doc.set_extension("subjectivity", getter=lambda doc: doc._.blob.sentiment.subjectivity)  
  
if not Doc.has_extension("assessments"):  
    Doc.set_extension("assessments", getter=lambda doc: doc._.blob.sentiment_assessments.assessments)

1. Write code that extracts the article html from https://web.archive.org/web/20210327165005/https://hackaday.com/2021/03/22/how-laser-headlights-work/ and dumps it to a .pkl (or other appropriate file)

In [4]:
article_page = requests.get('https://web.archive.org/web/20210327165005/https://hackaday.com/2021/03/22/how-laser-headlights-work/')
article_html = article_page.text

with open('laser_headlights.pkl', 'wb') as f:
    pickle.dump(article_html, f)

2. Read in your article's html source from the file you created in question 1 and print it's text (use `.get_text()`)

In [8]:
with open('laser_headlights.pkl', 'rb') as f:
    article_html = pickle.load(f)

soup = BeautifulSoup(article_html, "html.parser")
article_element = soup.find("article")
article_text = article_element.get_text() if article_element else soup.get_text()

# print(article_text)

# Cut everything after "Leave a comment" if that exists
cutoff_phrase = "Leave a comment"
if cutoff_phrase in article_text:
    article_text = article_text.split(cutoff_phrase)[0]

article_text = article_text.strip()


3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels). Make sure to remove things we don't care about (punctuation, stopwords, whitespace).

In [9]:
doc = nlp(article_text)

def we_care_about(token):
    return (
        not token.is_space
        and not token.is_punct
        and not token.is_stop
        and token.is_alpha  # only alphabetic words
        and len(token.text) > 2  # ignore short junk like "at", "to"
    )


tokens = [token.text.lower() for token in doc if we_care_about(token)]

token_freq = Counter(tokens)

top_5 = token_freq.most_common(5)

print("Top 5 Most Common Tokens (excluding stopwords, punctuation, and whitespace):")
for word, count in top_5:
    print(f"{word}: {count}")


Top 5 Most Common Tokens (excluding stopwords, punctuation, and whitespace):
laser: 35
headlights: 19
headlight: 11
technology: 10
led: 10


4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels). Make sure to remove things we don't care about (punctuation, stopwords, whitespace).

5. Define the following methods:
    * `score_sentence_by_token(sentence, interesting_token)` that takes a sentence and a list of interesting token and returns the number of times that any of the interesting words appear in the sentence divided by the number of words in the sentence
    * `score_sentence_by_lemma(sentence, interesting_lemmas)` that takes a sentence and a list of interesting lemmas and returns the number of times that any of the interesting lemmas appear in the sentence divided by the number of words in the sentence
    
You may find some of the code from the in class notes useful; feel free to use methods (rewrite them in this cell as well).  Test them by showing the score of the first sentence in your article using the frequent tokens and frequent lemmas identified in question 3.

6. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

8. Which tokens and lexems would be ommitted from the lists generated in questions 3 and 4 if we only wanted to consider nouns as interesting words?  How might we change the code to only consider nouns? Put your answer in this Markdown cell (you can edit it by double clicking it).